##### Generating CSV Data and DF for Pedido and Item_Pedido

In [2]:
import pandas as pd
import csv
from random import shuffle, choice, randrange, random
from faker import Faker
from datetime import datetime, timedelta
import locale
import os

fake = Faker()

try:
    locale.setlocale(locale.LC_ALL, 'pt_BR')
except:
    locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil') 

#TODO Parse to PT-BR
def gen_datetime(min_year=1985, max_year=datetime.now().year):
    # generate a datetime in format yyyy-mm-dd hh:mm:ss.000000
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return start + (end - start) * random() 

tbl_id = [id for id in range(1, 8)]
id_unidade = tbl_id.copy()
vlr_pedido = [111, 122, 225, 1008, 409, 887, 77]
vlr_item = [2.87, 88.77, 77.65, 10.13, 14.8, 9.88, 1.22]
shuffle(id_unidade)
shuffle(vlr_pedido)
shuffle(vlr_item)

pedido = {
        "Id_Unidade": id_unidade, 
        "Id_Pedido":  tbl_id, 
        "Tipo_Pedido": [choice((1, 2)) for tipo_p in range(1, 8)],
        "Data_Pedido": [gen_datetime().strftime("%d/%m/%Y") for data_p in range(1, 8)],
        "Vlr_Pedido": vlr_pedido,
        "Endereco_Entrega": [str(fake.address()) for endereco in range(1, 8)],
        "Taxa_Entrega": [randrange(1, 10) * 2.75 for taxa_entrega in range(1, 8)],
        "Status": [choice((1, 2, 3)) for st in range(1, 8)]
        }

id_pedido = id_unidade.copy()
shuffle(id_pedido)
id_produto = id_pedido.copy()
shuffle(id_produto)

item_pedido = { 
        "Id_Pedido":  id_pedido, 
        "Id_Item_Pedido": tbl_id,
        "Id_Produto": id_produto,
        "Qtd": [choice([i for i in range(100)]) for qtd in range(1, 8)],
        "Vlr_Item": vlr_item,
        "Observacao": [None for taxa_entrega in range(1, 8)],
        }


df_pedido = pd.DataFrame(pedido)
df_item_pedido = pd.DataFrame(item_pedido)

df_pedido = df_pedido.astype({'Data_Pedido':'datetime64[ns]'})
df_pedido = df_pedido.astype({'Endereco_Entrega':'string'})
df_item_pedido = df_item_pedido.astype({'Observacao':'string'})

print("Pedido DF: ")
display(df_pedido)
display(df_pedido.dtypes)

print("Item Pedido DF: ")
display(df_item_pedido)
display(df_item_pedido.dtypes)

Pedido DF: 


C:\Users\Pablo\AppData\Local\Temp\ipykernel_8060\1294743847.py:61: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_pedido = df_pedido.astype({'Data_Pedido':'datetime64[ns]'})


,Id_Unidade,Id_Pedido,Tipo_Pedido,Data_Pedido,Vlr_Pedido,Endereco_Entrega,Taxa_Entrega,Status
0,2,1,1,1997-08-18,122,USNV Green FPO AA 02373,11.00,3
1,5,2,1,2002-12-11,111,"22773 Bryan Skyway Howardtown, NV 40093",2.75,1
2,3,3,2,2000-09-16,409,"808 Walsh Highway Michaelhaven, IL 57239",19.25,1
3,7,4,2,1990-10-12,887,"9031 Coleman Roads Johnnystad, GU 34082",22.00,1
4,6,5,1,2008-06-22,1008,"7000 Christopher Keys South Matthewside, MI 80069",16.50,2
5,4,6,2,2015-03-21,225,"1040 Jennings Circle Apt. 690 Port Noah, MN 14061",2.75,1
6,1,7,1,2000-06-14,77,"905 Dakota Roads Suite 721 Robinsonville, RI 1...",8.25,1


Id_Unidade                   int64
Id_Pedido                    int64
Tipo_Pedido                  int64
Data_Pedido         datetime64[ns]
Vlr_Pedido                   int64
Endereco_Entrega    string[python]
Taxa_Entrega               float64
Status                       int64
dtype: object

Item Pedido DF: 


,Id_Pedido,Id_Item_Pedido,Id_Produto,Qtd,Vlr_Item,Observacao
0,7,1,3,2,2.87,<NA>
1,2,2,7,52,9.88,<NA>
2,6,3,6,42,77.65,<NA>
3,4,4,5,81,88.77,<NA>
4,5,5,2,31,14.80,<NA>
5,1,6,1,79,10.13,<NA>
6,3,7,4,76,1.22,<NA>


Id_Pedido                  int64
Id_Item_Pedido             int64
Id_Produto                 int64
Qtd                        int64
Vlr_Item                 float64
Observacao        string[python]
dtype: object

##### Creation of CSVs

In [5]:
def pivot_table(table_values):
    values, temp_list = [], []
    for idx in range(len(list(table_values.values())[0])):
        for value in table_values.values():
            temp_list += [value[idx]]
        values.append(temp_list)
        temp_list = []
    return values

pedido_values, item_pedido_values = pivot_table(pedido), pivot_table(item_pedido)

os.remove(".\csv-tables\item_pedido.csv")
os.remove(".\csv-tables\pedido.csv")

def create_csv(name, table, table_values):
    with open(f"./csv-tables/{name}.csv", 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(table)
    
        for row in table_values:
            writer.writerow(row)

try:
    create_csv("pedido", pedido, pedido_values)
    create_csv("item_pedido", item_pedido, item_pedido_values)
except Exception as e:
    print(f"Erro: {e}. CSVs não criados.")
else:
    print("CSVs pedido e item_pedido criados.")

CSVs pedido e item_pedido criados.
